<a href="https://colab.research.google.com/github/SrijaG29/Pyspark_practise/blob/main/PySpark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**PySpark**

In [2]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488490 sha256=14f6ac69297a2bcfacf2ec73d44fdbffdbaf3f3b438e780403fac7b3ab65bbd7
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [2]:
from pyspark import SparkConf,SparkContext

In [3]:
conf = SparkConf().setAppName("Read File")
sc = SparkContext.getOrCreate(conf = conf)

Instead of dowloading the text file using path of the file we can directly create the text file here.

For writing the file.

In [20]:
with open('sampletext.txt', 'w') as f:
  f.write('Hi How are you\n')
  f.write('Hope you are doing good\n')
  f.write('great')

For reading the file.

In [21]:
with open('sampletext.txt','r') as f:
  x = f.read()
  print(x)

Hi How are you
Hope you are doing good
great


Fo direct download from the files in colab we need to give file path.

In [22]:
text = sc.textFile('/content/sampletext.txt')
text.collect()

['Hi How are you', 'Hope you are doing good', 'great']

This is another way to write or creat rdd.

In [7]:
temp = sc.parallelize([['a','b','c'],['d','e','f']])
temp.collect()

[['a', 'b', 'c'], ['d', 'e', 'f']]

**Map**
It is used as a maper of data from one state to another.

In [23]:
rdd = text.map(lambda x : x.split(' '))

In [24]:
rdd.collect()

[['Hi', 'How', 'are', 'you'],
 ['Hope', 'you', 'are', 'doing', 'good'],
 ['great']]

In [25]:
def ex(x):
  l = x.split()
  f = []
  for i in l:
    f.append(i+'hello')
  return f
rdd1 = text.map(ex)

In [26]:
rdd1.collect()

[['Hihello', 'Howhello', 'arehello', 'youhello'],
 ['Hopehello', 'youhello', 'arehello', 'doinghello', 'goodhello'],
 ['greathello']]

find the length of each word and return only length.

In [27]:
def findlength(x):
  l = x.split(' ')
  f = ''
  for i in range(len(l)):
    if i!= len(l)-1:
      f = f + str(len(l[i])) + ' '
    else:
      f = f + str(len(l[i]))
  return f
rdd = text.map(findlength)

In [28]:
rdd.collect()

['2 3 3 3', '4 3 3 5 4', '5']

**flatMap:**
 It is used as a mapper of data and explodes data before final output.

In [29]:
rdd1 = rdd.flatMap(lambda x : x.split(' '))
rdd1.collect()

['2', '3', '3', '3', '4', '3', '3', '5', '4', '5']

**filter:** It is used to remove elements from rdd.

In [30]:
rdd2 = rdd1.filter(lambda x : x!='2')
rdd2.collect()

['3', '3', '3', '4', '3', '3', '5', '4', '5']

In [31]:
def ex(x):
  if x != '2':
    return True
  else:
    return False
rdd2 = rdd1.filter(ex)
rdd2.collect()

['3', '3', '3', '4', '3', '3', '5', '4', '5']

**Distinct:** It is used to get unique/distinct elements

In [32]:
def ex(x):
  if x != '2':
    return True
  else:
    return False
rdd2 = rdd1.filter(ex)
rdd3 = rdd2.distinct()
rdd3.collect()

['4', '3', '5']

**Count:** It return no.of elemets in rdd.

In [33]:
rdd2.count()

9

**CountByValue:** It provides how many times each value hass occur in rdd.

In [34]:
rdd2.countByValue()

defaultdict(int, {'3': 5, '4': 2, '5': 2})

**GroupByValue:** It is used to create groups based on Keys in RDD.

In [35]:
def findlen(x):
  return (x,len(x))
rdd4 = rdd1.map(findlen)
rdd4.groupByKey().mapValues(list).collect()

[('4', [1, 1]), ('2', [1]), ('3', [1, 1, 1, 1, 1]), ('5', [1, 1])]

**ReduceByKey:** It is used to combine data based on keys in rdd.

In [36]:
rdd4.reduceByKey(lambda x,y:x+y).collect()

[('4', 2), ('2', 1), ('3', 5), ('5', 2)]

**Spark SQL**

If we read data from SprakContext then spark provides data as RDD.

If we read data from SpaekSession then spark provides data as dataframes.

In [3]:
from pyspark import SparkContext

In [9]:
from pyspark.sql import SparkSession
spark = SparkSession.builder\
        .appName("py-Spark")\
        .getOrCreate()

In [6]:
!wget https://raw.githubusercontent.com/haneeth25/Libraries_prac/main/Student_performance_data%20_.csv

--2024-07-25 09:29:43--  https://raw.githubusercontent.com/haneeth25/Libraries_prac/main/Student_performance_data%20_.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 166901 (163K) [text/plain]
Saving to: ‘Student_performance_data _.csv’

Student_performance 100%[===================>] 162.99K  --.-KB/s    in 0.006s  

2024-07-25 09:29:43 (28.0 MB/s) - ‘Student_performance_data _.csv’ saved [166901/166901]



Here we are creating dataframe df by using csv file "student_performance_data".

In [17]:
df = spark.read.option("header",True).csv('/content/Student_performance_data.csv')

In [18]:
df.show()

+---------+---+------+---------+-----------------+------------------+--------+--------+---------------+---------------+------+-----+------------+------------------+----------+
|StudentID|Age|Gender|Ethnicity|ParentalEducation|   StudyTimeWeekly|Absences|Tutoring|ParentalSupport|Extracurricular|Sports|Music|Volunteering|               GPA|GradeClass|
+---------+---+------+---------+-----------------+------------------+--------+--------+---------------+---------------+------+-----+------------+------------------+----------+
|     1001| 17|     1|        0|                2|19.833722807854713|       7|       1|              2|              0|     0|    1|           0| 2.929195591667681|       2.0|
|     1002| 18|     0|        0|                1| 15.40875605584674|       0|       0|              1|              0|     0|    0|           0| 3.042914833436377|       1.0|
|     1003| 15|     0|        2|                3|  4.21056976881226|      26|       0|              2|              0| 

We will print Schema of the data frame by default it will take all datatupes as strings.

In [19]:
df.printSchema()

root
 |-- StudentID: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Ethnicity: string (nullable = true)
 |-- ParentalEducation: string (nullable = true)
 |-- StudyTimeWeekly: string (nullable = true)
 |-- Absences: string (nullable = true)
 |-- Tutoring: string (nullable = true)
 |-- ParentalSupport: string (nullable = true)
 |-- Extracurricular: string (nullable = true)
 |-- Sports: string (nullable = true)
 |-- Music: string (nullable = true)
 |-- Volunteering: string (nullable = true)
 |-- GPA: string (nullable = true)
 |-- GradeClass: string (nullable = true)



If we want it to automatically take the datatype as the datatype in csv then we use this way.

In [16]:
df = spark.read.option('inferSchema',True).option('header',True).csv('/content/Student_performance_data.csv')
# df.show()
df.printSchema()

root
 |-- StudentID: integer (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Gender: integer (nullable = true)
 |-- Ethnicity: integer (nullable = true)
 |-- ParentalEducation: integer (nullable = true)
 |-- StudyTimeWeekly: double (nullable = true)
 |-- Absences: integer (nullable = true)
 |-- Tutoring: integer (nullable = true)
 |-- ParentalSupport: integer (nullable = true)
 |-- Extracurricular: integer (nullable = true)
 |-- Sports: integer (nullable = true)
 |-- Music: integer (nullable = true)
 |-- Volunteering: integer (nullable = true)
 |-- GPA: double (nullable = true)
 |-- GradeClass: double (nullable = true)



If we want to assign datatypes manually then we need to use this.

for this first we need to import StuctType,StructField,StringType,IntegerType.

In [ ]:
schema = StructType(
    [
        StructField('age',IntegerType(),True),
        StructField('Gender',IntegerType(),True),
        StructField('age',IntegerType(),True),
    ]
)